# Backtesting

In [ ]:
# Load Libraries
import os
import pandas as pd
import numpy as np
import talib as ta
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime as dt
import hvplot.pandas
import ccxt
%matplotlib inline

In [ ]:
# Import tickers from screener 

tickers = ["BBY", "UAL", "DISCK", "FANG", "NEE"]

In [ ]:
load_dotenv()
# Import API Variable Keys
## Stocks
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [ ]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Format current date as ISO format
#today = pd.Timestamp(now, tz="America/New_York").isoformat()

# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "15Min"
limit=50
end_date = pd.Timestamp.now(tz="America/New_York").isoformat()


df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    limit,
    end = end_date
).df

df_portfolio.index = df_portfolio.index.date
df_portfolio = df_portfolio.dropna()
df_portfolio.tail()

In [ ]:
# Create Datafrom for solo ticker
df_bby = pd.DataFrame()
df_bby = df_portfolio['BBY']
df_bby.tail()

In [ ]:
# Create function that applies Tech Analysis
def applyTA (input_df):
    """
        Add columns of technical analysis in the dataframe
        Parameter:
            input_df - dataframe with date index and columns of OHLCV data
        Returns:
            df_ta - dataframe with date index and technical analysis signals
    """
    
    df = input_df.copy()
    
    # Construct EMA Singals
    df['8D_EMA'] = ta.EMA(df['close'], timeperiod = 8)
    df['21D_EMA'] = ta.EMA(df['close'], timeperiod = 21)
    df['50D_EMA'] = ta.EMA(df['close'], timeperiod = 50)
    df['8_21_x_long'] = np.where(df['8D_EMA'] > df['21D_EMA'], 1.0, 0.0)
    df['8_21_x_short'] = np.where(df['8D_EMA'] < df['21D_EMA'], -1.0, 0.0)
    df['8_21_x_signal'] = df['8_21_x_long'] + df['8_21_x_short'] 
    df['21_50_x_long'] = np.where(df['21D_EMA'] > df['50D_EMA'], 1.0, 0.0)
    df['21_50_x_short'] = np.where(df['21D_EMA'] < df['50D_EMA'], -1.0, 0.0)
    df['21_50_x_signal'] = df['21_50_x_long'] + df['21_50_x_short']
    df['above_8D'] = np.where(df['close'] > df['8D_EMA'], 1.0, 0.0)
    df['above_21D'] = np.where(df['close'] > df['21D_EMA'], 1.0, 0.0)
    df['above_50D'] = np.where(df['close'] > df['50D_EMA'], 1.0, 0.0)
    
    # Bollinger Bands
    df['bb_upper_band'], df['bb_middle_band'], df['bb_lower_band'] = ta.BBANDS(df['close'], timeperiod =20)
    df['bb_long'] = np.where(df['close'] < df['bb_lower_band'], 1.0, 0.0)
    df['bb_short'] = np.where(df['close'] > df['bb_upper_band'], -1.0, 0.0)
    df['bb_signal'] = df['bb_long'] + df['bb_short']
    
    # Calculate Stochastic Oscillators trading signal
    df['slowk'], df['slowd'] = ta.STOCH(df['high'], df['low'], df['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)   
    df['slowk_x_long'] = np.where(df['slowk'] > df['slowd'], 1.0, 0.0)
    df['slowk_x_short'] = np.where(df['slowk'] < df['slowd'], -1.0, 0.0)
    df['slowk_x_signal'] = df['slowk_x_long'] + df['slowk_x_short']
    
     # Calculate ADX
    df['adx'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['adx_strong'] = np.where(df['adx'] > 50, 1.0, 0.0)
    df['adx_weak'] = np.where(df['adx'] < 20, 1.0, 0.0)

    # Calculate OBV signal
    df['obv'] = ta.OBV(df['close'], df['volume'])
    df['obv_change']=df['obv'].pct_change()
    df['obv_increase'] = np.where(df['obv_change'] > 0, 1.0, 0.0)

    # Calculate Chaikin A/D Oscillator signal
    df['ADOSC'] = ta.ADOSC(df['high'], df['low'], df['close'], df['volume'], fastperiod=3, slowperiod=10)    
    df['adosc_positive'] = np.where(df['ADOSC'] > 0, 1.0, 0.0)

    # Drop non-TA signals columns and NAs
    df_ta = df[{'8_21_x_signal', '21_50_x_signal', 'above_8D', 'above_21D', 'above_50D', 'bb_signal', 'slowk_x_signal', 'adx_strong', 'adx_weak', 'obv_increase', 'adosc_positive'}]
    df_ta = df_ta.dropna()
    
    return df_ta


In [ ]:
# Apply Tech Analysis to chosen ticker
applyTA(df_bby).tail()

In [ ]:
# Create Dataframe
df_TA_bby = applyTA(df_bby)
df_TA_bby.tail()

In [ ]:
# Create loop to find signals for every 15 minutes
df_TA_bby.reset_index(inplace=True)

for row in range(len(df_TA_bby)):
    df_TA_bby.loc[row, 'total_score'] = df_TA_bby.loc[row, '8_21_x_signal'] + df_TA_bby.loc[row, '21_50_x_signal'] + df_TA_bby.loc[row,'above_8D'] + df_TA_bby.loc[row, 'above_21D'] + df_TA_bby.loc[row,'above_50D'] + df_TA_bby.loc[row,'bb_signal'] + df_TA_bby.loc[row,'slowk_x_signal'] + df_TA_bby.loc[row,'adx_strong'] + df_TA_bby.loc[row,'adx_weak'] + df_TA_bby.loc[row,'obv_increase'] + df_TA_bby.loc[row,'adosc_positive']
df_TA_bby.tail()

In [ ]:
# Create DataFrame for total scores
df_TA_bby = df_TA_bby.drop(columns=['above_21D', 'above_50D', 'obv_increase', 'bb_signal', 'adx_weak', 'adosc_positive', 'adx_strong', 'above_8D', '8_21_x_signal', '21_50_x_signal', 'slowk_x_signal'])
df_TA_bby.tail()

In [ ]:
# Create DataFrame for Closing prices per interval
df_bby = df_bby.drop(columns=['open', 'high', 'low', 'volume'])
df_TA_bby.index = df_TA_bby['index']
df_TA_bby = df_TA_bby.drop(columns=['index'])
df_TA_bby.index.name = 'date'
df_bby.index.name = 'date'
df_TA_bby.tail()

In [ ]:
# Concat closing price and total_score DataFrames
df_concat = pd.concat([df_bby , df_TA_bby], axis='columns', join='outer')
df_concat = df_concat.reset_index()
df_concat.tail()

In [ ]:
# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
#signals_df["Signal"][short_window:] = np.where(
#    signals_df["SMA50"][short_window:] < signals_df["SMA100"][short_window:], 1.0, 0.0
#)

#df_concat['signal'] = 0.0
for rows in range(len(df_concat["date"])):
    if df_concat.loc[rows, 'total_score'] >=3:
        df_concat.loc[rows, "signal"] = 1
    elif df_concat.loc[rows, 'total_score'] <= -3:
        df_concat.loc[rows, "signal"] = -1
    else:
        df_concat.loc[rows, "signal"] = 0

# Calculate the points in time at which a position should be taken, 1 or -1
df_concat["Entry/Exit"] = df_concat["signal"].diff()

# Print the DataFrame
df_concat.dropna(inplace=True)
df_concat

In [ ]:
# Visualize exit position relative to close price
entry = df_concat[df_concat["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color="green", legend=False, ylabel="Price in $", width=1000, height=400
)

# Visualize entry position relative to close price
exit = df_concat[df_concat["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color="red", legend=False, ylabel="Price in $", width=1000, height=400
)

# Visualize close price for the investment
security_close = df_concat["close"].hvplot(
    line_color="lightgray", ylabel="Price in $", width=1000, height=400
)

# Overlay plots
entry_exit_plot = security_close * entry * exit
entry_exit_plot

In [ ]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = -500

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
df_concat["Position"] = share_size * df_concat["signal"]

# Find the points in time where a 500 share position is bought or sold
df_concat["Entry/Exit Position"] = df_concat["Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
df_concat["Portfolio Holdings"] = (
    df_concat["close"] * df_concat["Entry/Exit Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
df_concat["Portfolio Cash"] = (
    initial_capital - (df_concat["close"] * df_concat["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
df_concat["Portfolio Total"] = (
    df_concat["Portfolio Cash"] + df_concat["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
df_concat["Portfolio Daily Returns"] = df_concat["Portfolio Total"].pct_change()

# Calculate the cumulative returns
df_concat["Portfolio Cumulative Returns"] = (
    1 + df_concat["Portfolio Daily Returns"]
).cumprod() - 1
#Print DF
df_concat

In [ ]:
# Visualize exit position relative to total portfolio value
entry = df_concat[df_concat["Entry/Exit"] == -1.0]["Portfolio Total"].hvplot.scatter(
    color="red", legend=False, ylabel="Total Portfolio Value", width=1000, height=400
)

# Visualize entry position relative to total portfolio value
exit = df_concat[df_concat["Entry/Exit"] == 1.0]["Portfolio Total"].hvplot.scatter(
    color="green", legend=False, ylabel="Total Portfolio Value", width=1000, height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = df_concat[["Portfolio Total"]].hvplot(
    line_color="lightgray", ylabel="Total Portfolio Value", width=1000, height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)